# Second virial coefficient calculation of a Trimer using Mayer-Sampling


Here, we reproduce the virial coefficient of a trimer at the Boyle temperature as reported Table 1 of the following manuscript for $L=0.4\sigma$

https://dx.doi.org/10.1063/1.4918557

In [1]:
import unittest
import feasst as fst

class TestMayerSamplingTrimer(unittest.TestCase):
    def test(self):
        mc = fst.MonteCarlo()
        config = fst.MakeConfiguration(fst.args({"cubic_box_length": str(fst.NEAR_INFINITY)}))
        config.add_particle_type(fst.install_dir() + "/forcefield/trimer_0.4L.fstprt")
        config.add_particle_type(fst.install_dir() + "/forcefield/trimer_0.4L.fstprt", "2")
        config.add_particle_of_type(0)
        config.add_particle_of_type(1)
        rwca = 2.**(1./6.)
        config.set_model_param("cutoff", 0, 1, rwca)
        config.set_model_param("cutoff", 0, 3, rwca)
        config.set_model_param("cutoff", 1, 2, rwca)
        config.set_model_param("cutoff", 2, 3, rwca)
        mc.add(config)
        mc.add(fst.MakePotential(fst.MakeLennardJonesForceShift()))
        ref = fst.MakePotential(fst.MakeHardSphere())
        ref.set(ref.model_params(mc.system().configuration()).deep_copy())
        ref.set_model_param("sigma", 0, 1.)
        ref.set_model_param("sigma", 1, 0)
        ref.set_model_param("sigma", 2, 1.)
        ref.set_model_param("sigma", 3, 0)
        mc.add_to_reference(ref)
        mc.set(fst.MakeThermoParams(fst.args({"beta": str(1./0.815)})))
        mayer = fst.MakeMayerSampling()
        mc.set(mayer)
        mc.add(fst.MakeTrialTranslate(fst.args({"new_only": "true", "reference_index": "0",
            "tunable_param": "1", "particle_type": "1"})))
        mc.add(fst.MakeTrialRotate(fst.args({"new_only": "true", "reference_index": "0",
            "tunable_param": "40"})))
        steps_per = "1e4"
        mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": steps_per, "file_name": "tmp/trib"})))
        mc.attempt(int(1e6))
        b2hs = 2./3.*fst.PI*mc.configuration().model_params().select("sigma").value(0)**3
        print(mayer.mayer().str())
        self.assertAlmostEqual(0, mayer.mayer().average(), delta=4*mayer.mayer().block_stdev())


If the test passes, the energy is within the tolerance of the SRSW value and the two ensemble average methods agreed.

In [2]:
%time  # Note: any line starting with % is only to be used with ipynb
unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestMayerSamplingTrimer) ... 

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
average,stdev,block_stdev,moment0,moment1,moment2,moment3,moment4,
-0.00036999999999999999,1.0000004315503384,0.0028870451426098215,1000000,-370,1000000,-370,1000000,


ok

----------------------------------------------------------------------
Ran 1 test in 3.943s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!